In [29]:
from openai import OpenAI
from datetime import datetime
import pandas as pd
import pyperclip
import numpy as np
import pinecone
import torch.nn as nn
import ast
import matplotlib.pyplot as plt
import torch
import esm
from datetime import datetime
import pandas as pd
import textwrap

In [30]:
client = OpenAI(api_key="<YOUR KEY HERE>")
pinecone.init(api_key="<YOUR KEY HERE>", environment="us-east4-gcp")
textIndex = pinecone.Index('text-embeddings')
esmIndex = pinecone.Index('esm-embeddings')
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()
print()

In [31]:
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.fc1 = nn.Linear(1536, 768)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(768, 1280)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
mseModel = EmbeddingNet()
cosModel = EmbeddingNet()
mseModel.load_state_dict(torch.load('PATH TO PTH MODELS HERE'))
cosModel.load_state_dict(torch.load('PATH TO PTH MODELS HERE'))
mseModel.eval()
cosModel.eval()
W = np.load('Wnorm.npy')
print("Loaded models")

Loaded models


In [32]:
def query(embedding, index, k):
    result = index.query(
      vector=embedding,
      top_k=k,
      include_metadata=True
    )
    return result.matches

def textSearch(text, k = 50):
    embedding = client.embeddings.create(input=text, model="text-embedding-ada-002").data[0].embedding
    
    return query(embedding, textIndex, k)

def esmSearch(sequence, k = 50):
    data = [("protein1", sequence),]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33]
    embedding = token_representations[0, 1: len(sequence) + 1].mean(0).detach().numpy().tolist()
    
    return query(embedding, esmIndex, k)
    
# mode 0 - matrix. mode 1 - mse + cos loss. mode 2 - mse loss only
def textESMSearch(text, mode = 0, k = 50):
    embedding = client.embeddings.create(input=text, model="text-embedding-ada-002").data[0].embedding
    transformed_embedding = []
    if mode == 0:
        transformed_embedding = np.dot(embedding, W).tolist()
    elif mode == 1:
        embedding_tensor = torch.tensor(embedding).float().unsqueeze(0)
        transformed_embedding = cosModel(embedding_tensor).detach().numpy().squeeze().tolist()
    else:
        embedding_tensor = torch.tensor(embedding).float().unsqueeze(0)
        transformed_embedding = mseModel(embedding_tensor).detach().numpy().squeeze().tolist()
    
    return query(transformed_embedding, esmIndex, k)

# JSON Object
def display(res):
    for protein in res:
        print(protein.metadata['Name'], '(' + str(protein.score)+')', '    |    ', "ID:", protein.id)
        
        description = protein.metadata['Description']
        formatted_description = textwrap.fill(description, width=100, initial_indent='     ',subsequent_indent='   ')
        print(formatted_description)
        print()

In [63]:
res = textSearch("Cytokine involved with immune system that targets blood cells")
# res = textSearch("Component of a Polycomb group (PcG) multiprotein PRC1-like complex, a complex class required to maintain the transcriptionally repressive state of many genes, including Hox genes, throughout development. PcG PRC1 complex acts via chromatin remodeling and modification of histones; it mediates monoubiquitination of histone H2A 'Lys-119', rendering chromatin heritably changed in its expressibility. The PRC1-like complex that contains PCGF5, RNF2, CSNK2B, RYBP and AUTS2 has decreased histone H2A ubiquitination activity, due to the phosphorylation of RNF2 by CSNK2B. As a consequence, the complex mediates transcriptional activation (By similarity). In the cytoplasm, plays a role in axon and dendrite elongation and in neuronal migration during embryonic brain development. Promotes reorganization of the actin cytoskeleton, lamellipodia formation and neurite elongation via its interaction with RAC guanine nucleotide exchange factors, which then leads to the activation of RAC1")
display(res)

# res = esmSearch("MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAPSLASSSGSDKEDNGKPPSSAPSRPRPPRRKRRESTSAEEDIIDGFAMTSFVTFEALEKDVAVKPQERAEKRQTPLTKKKREALTNGLSFHSKKSRLSHSHHYSSDRENDRNLCQHLGKRKKMPKGLRQLKPGQNSCRDSDSESASGESKGFQRSSSRERLSDSSAPSSLGTGYFCDSDSDQEEKASDASSEKLFNTVLVNKDPELGVGALPEHNQDAGPIVPKISGLERSQEKSQDCCKEPVFEPVVLKDPHPQLPQLPSQAQAEPQLQIPSPGPDLVPRTEAPPQFPPPSTQPAQGPPEAQLQPAPLPQVQQRPPRPQSPSHLLQQTLPPVQSHPSSQSLSQPLSAYNSSSLSLNSLSSRSSTPAKTQPAPPHISHHPSASPFPLSLPNHSPLHSFTPTLQPPAHSHHPNMFAPPTALPPPPPLTSGSLQVPGHPAGSTYSEQDILRQELNTRFLASQSADRGASLGPPPYLRTEFHQHQHQHQHTHQHTHQHTFTPFPHAIPPTAIMPTPAPPMFDKYPTKVDPFYRHSLFHSYPPAVSGIPPMIPPTGPFGSLQGAFQPKTSNPIDVAARPGTVPHTLLQKDPRLTDPFRPMLRKPGKWCAMHVHIAWQIYHHQQKVKKQMQSDPHKLDFGLKPEFLSRPPGPSLFGAIHHPHDLARPSTLFSAAGAAHPTGTPFGPPPHHSNFLNPAAHLEPFNRPSTFTGLAAVGGNAFGGLGNPSVTPNSVFGHKDSPSVQNFSNPHEPWNRLHRTPPSFPTPPPWLKPGELERSASAAAHDRDRDVDKRDSSVSKDDKERESVEKRHPSHPSPAPPVPVSALGHNRSSTDPTTRGHLNTEAREKDKPKEKERDHSGSRKDLTTEEHKAKESHLPERDGHSHEGRAAGEEPKQLSRVPSPYVRTPGVDSTRPNSTSSREAEPRKGEPAYENPKKNAEVKVKEERKEDHDLPTEAPQAHRTSEAPPPSSSASASVHPGPLASMPMTVGVTGIHAMNSIGSLDRTRMVTPFMGLSPIPGGERFPYPSFHWDPMRDPLRDPYRDLDMHRRDPLGRDFLLRNDPLHRLSTPRLYEADRSFRDREPHDYSHHHHHHHHPLAVDPRREHERGGHLDERERLHVLREDYEHPRLHPVHPASLDGHLPHPSLLTPGLPSMHYPRISPTAGHQNGLLNKTPPTAALSAPPPLISTLGGRPGSPRRTTPLSAEIRERPPSHTLKDIEAR")
# text = "Protein that interacts with immunoglobulin. Involved with digestive system."
# text = "Cytokine involved with humoral immune responses"  # works well
# text = "Involved in mitochondrial respiratory chain. Does synthase "  # works well
# text = "Acyl-CoA dehydrogenase only active with R- and S-2-methyl-C15-CoA"

Granulocyte-macrophage colony-stimulating factor (0.875631571)     |     ID: P01587
     Cytokine that stimulates the growth and differentiation of hematopoietic precursor cells from
   various lineages, including granulocytes, macrophages, eosinophils and erythrocytes

Interleukin-7 (0.870867252)     |     ID: P10168
     Hematopoietic cytokine that plays an essential role in the development, expansion, and survival
   of naive and memory T-cells and B-cells thereby regulating the number of mature lymphocytes and
   maintaining lymphoid homeostasis (PubMed:7699333, PubMed:28811625). Mechanistically, exerts its
   biological effects through a receptor composed of IL7RA subunit and the cytokine receptor common
   subunit gamma/CSF2RG. Binding to the receptor leads to activation of various kinases including
   JAK1 or JAK3 depending on the cell type and subsequently propagation of signals through
   activation of several downstream signaling pathways including the PI3K/Akt/mTOR or the JA

In [65]:
seq = "MTPMRKINPLMKLINHSFIDLPTPSNISAWWNFGSLLGACLILQITTGLFLAMQYSPDAWTAFSSIAHITRDVNYGWIIRYLHANGASMFFICLFLHIGRGLYYGSFLYSETWNIGIILLLATMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTDLVQWIWGGYSVDSPTLTRFFTFHFILPFIIAALAALHLLFLHETGSNNPLGITSHSDKITFHPYYTIKDALGLLLFLLSLMTLTLFSPDLLGDPDNYTLANPLNTPPHIKPEWYFLFAYTILRSVPNKLGGVLALLLSILILAMIPILHMSKQQSMMFRPLSQSLYWLLAADLLILTWIGGQPVSYPFTIIGQVASVLYFTTILILMPTISLIENKMLKWA"
# ^^ Cytochrome b in humans: https://www.ebi.ac.uk/interpro/protein/UniProt/A0A023I7F4/
print(len(seq))
res = esmSearch(seq[:])
display(res)

380
Cytochrome b (0.996476293)     |     ID: P00158
     Component of the ubiquinol-cytochrome c reductase complex (complex III or cytochrome b-c1
   complex) that is part of the mitochondrial respiratory chain. The b-c1 complex mediates electron
   transfer from ubiquinol to cytochrome c. Contributes to the generation of a proton gradient
   across the mitochondrial membrane that is then used for ATP synthesis

Protein transport protein Sec61 subunit alpha isoform 1 (0.959146738)     |     ID: P61620
     Component of SEC61 channel-forming translocon complex that mediates transport of signal
   peptide-containing precursor polypeptides across the endoplasmic reticulum (ER). Forms a ribosome
   receptor and a gated pore in the ER membrane, both functions required for cotranslational
   translocation of nascent polypeptides. May cooperate with auxiliary protein SEC62, SEC63 and
   HSPA5/BiP to enable post-translational transport of small presecretory proteins. The SEC61
   channel is al

In [36]:
# res = textESMSearch("Cytokine involved with humoral immune responses")
res = textESMSearch("Involved with transport to cell membrane")
# res = textSearch("Involved with transport to cell membrane")
display(res)

Glycerophosphodiester phosphodiesterase domain-containing protein 5 (0.982093632)     |     ID: Q640M6
     Glycerophosphodiester phosphodiesterase that promotes neurite formation and drives spinal motor
   neuron differentiation (PubMed:17275818, PubMed:18667693). Mediates the cleavage of
   glycosylphosphatidylinositol (GPI) anchor of target proteins: removes the GPI-anchor of RECK,
   leading to release RECK from the plasma membrane (By similarity). May contribute to the osmotic
   regulation of cellular glycerophosphocholine

Protein SERAC1 (0.979960322)     |     ID: Q3U213
     Plays an important role in the phosphatidylglycerol remodeling that is essential for both
   mitochondrial function and intracellular cholesterol trafficking. May catalyze the remodeling of
   phosphatidylglycerol and be involved in the transacylation-acylation reaction to produce
   phosphatidylglycerol-36:1. May be involved in bis(monoacylglycerol)phosphate biosynthetic pathway
   (By similarity)

Phosph